# Лабораторна робота №2. Етап розуміння даних за CRISP-DM.

###1. Завантажте набір даних через Colab у DataFrame за допомогою pandas і відобразіть перші 10 записів.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_fifa23.csv')
df.head(10)

MessageError: Error: credential propagation was unsuccessful

###2. Опишіть колонки, які ви плануєте використовувати для аналізу в подальшому. Що за дані в них зберігаються?

Серед специфічних колонок та тих, що потребують пояснення предметної області, можна виділити такі:
*   value_eur – ринкова (трансферна) вартість гравця в євро;
*   player_positions – позиція(ї) гравця на полі (категоріальна змінна, може містити кілька позицій);
*   age – вік гравця;
*   height_cm – зріст гравця;
*   weight_kg – вага гравця;
*   overall – загальний рейтинг гравця у FIFA (інтегральна оцінка рівня);
*   pace – швидкість;
*   shooting – удар/завершення атак;
*   passing – точність пасу.
*   dribbling – дриблінг.
*   defending – захисні навички.
*   physic – фізика/боротьба/витривалість.

In [ ]:
selected_columns = [
    'value_eur',
    'player_positions',
    'age',
    'height_cm',
    'weight_kg',
    'overall',
    'pace',
    'shooting',
    'passing',
    'dribbling',
    'defending',
    'physic'
]

df_selected = df[selected_columns].copy()
df_selected = df_selected[~df_selected['player_positions'].str.contains('GK', na=False)].copy()
df_selected.head()

###3. Які типи даних є у вашому наборі даних? Чи є категоріальні, числові дані? Виведіть усі варіанти категорій для певної колонки.
У моєму наборі даних представлені такі типи:
*   Числові (numeric): value_eur, age, height_cm, weight_kg, overall, pace, shooting, passing, dribbling, defending, physic
*   Категоріальні (categorical / text): player_positions

Позиції гравців записуються скорочено. Типові скорочення у FIFA:
* CB (Centre Back) – центральний захисник
* LB / RB (Left Back / Right Back) – лівий / правий захисник
* LWB / RWB (Left Wing Back / Right Wing Back) – лівий / правий вінгбек
* CDM (Central Defensive Midfielder) – опорний півзахисник
* CM (Central Midfielder) – центральний півзахисник
* CAM (Central Attacking Midfielder) – атакувальний півзахисник
* LM / RM (Left Midfielder / Right Midfielder) – лівий / правий півзахисник
* LW / RW (Left Winger / Right Winger) – лівий / правий вінгер
* CF (Centre Forward) – відтягнутий форвард
* ST (Striker) – нападник





In [ ]:
df_selected.dtypes
df_selected['player_positions'].nunique()

positions = (
    df_selected['player_positions']
    .dropna()
    .str.split(', ')
    .explode()
)

positions.unique()
positions.value_counts()


###4. Виведіть максимальні і мінімальні значення для числових колонок. Чи вони коректні в реаліях домену?

In [ ]:
df_selected.select_dtypes(include=['int64', 'float64']).agg(['min', 'max'])

###5. Чи є у вас пусті дані, null значення, нулі або інакшим чином некоректні записи?

In [ ]:
df_selected.isnull().sum()
(df_selected.isnull().sum() / len(df_selected)) * 100


###6. Виведіть 3 або більше розподілів для даних. Який тип розподілу для них найближчий?

*   value_eur - показує, як розподіляється ринкова вартість гравців у вибірці. Дає розуміння, чи більшість гравців дешеві або дорогі, і наскільки є нерівномірність між масовими гравцями та зірками.
*   overall - показує загальний рівень гравців у датасеті. Дозволяє оцінити, який рейтинг є типовим, та наскільки сильно гравці відхиляються від середнього рівня.
*   age - показує вікову структуру гравців. Дає уявлення про те, у якому віковому діапазоні зосереджена основна маса футболістів і наскільки представлена старша або молодша група.
*   value_eur (log transform) - показує ту саму ринкову вартість, але в масштабі, який дозволяє побачити реальну форму розподілу без впливу екстремально великих значень. Дає змогу коректніше аналізувати структуру вартості для статистичного моделювання.

In [ ]:
# 1 value_eur – Lognormal distribution
plt.figure(figsize=(5,3))
sns.histplot(
    np.log10(df_selected[df_selected['value_eur'] > 0]['value_eur']),
    bins=50
)
plt.title('Normal distribution after log transform (value_eur)')
plt.xlabel('log10(value_eur)')
plt.show()


# 2 overall – Normal distribution
plt.figure(figsize=(5,3))
sns.histplot(df_selected['overall'], bins=30)
plt.title('Normal distribution (overall)')
plt.show()


# 3 age – Not normal distribution
plt.figure(figsize=(5,3))
sns.histplot(df_selected['age'], bins=range(15, 46))
plt.title('Not normal distribution (age)')
plt.show()



###7. Виведіть 3 (або більше) графіки для ваших колонок.

In [ ]:
# Графік 1: Top 15 Most Common Player Positions
positions = (
    df_selected['player_positions']
    .dropna()
    .str.split(', ')
    .explode()
)

top_positions = positions.value_counts().head(15)

plt.figure(figsize=(10,6))
sns.barplot(x=top_positions.values, y=top_positions.index)
plt.title('Top 15 Most Common Player Positions')
plt.xlabel('Number of Players')
plt.ylabel('Position')
plt.show()


# Графік 2: Market Value by Primary Position (Log Scale)
df_selected['primary_position'] = df_selected['player_positions'].str.split(', ').str[0]

top_primary = df_selected['primary_position'].value_counts().head(10).index
df_pos_top = df_selected[df_selected['primary_position'].isin(top_primary)]

plt.figure(figsize=(12,6))
sns.boxplot(data=df_pos_top, x='primary_position', y='value_eur')
plt.yscale('log')
plt.title('Market Value by Primary Position (Log Scale)')
plt.xlabel('Primary Position')
plt.ylabel('Market Value (log scale)')
plt.show()


# Графік 3: Average Overall Rating by Position.
avg_overall = (
    df_pos_top
    .groupby('primary_position')['overall']
    .mean()
    .sort_values(ascending=False)
)

plt.figure(figsize=(10,6))
sns.barplot(x=avg_overall.index, y=avg_overall.values)
plt.title('Average Overall Rating by Position (Top 10)')
plt.xlabel('Position')
plt.ylabel('Average Overall Rating')
plt.show()


###8. Виведіть кореляційну матрицю або heatmap для ваших даних.

In [ ]:
numeric_df = df_selected.select_dtypes(include=['int64', 'float64'])
corr = numeric_df.corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True)
plt.title('Correlation matrix (selected columns)')
plt.show()
